# Liquour Licenses - Regression 

In [22]:
# Import libraries

import numpy as np
import pandas as pd
from math import pi

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression as lr
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

In [23]:
# Read the clean dataset

df = pd.read_csv('ll_cleaned.csv')

In [24]:
df

,LicenseClass,SubClass,LicenseNumber,LicenseDate,LicenseEndDate,LicenseYear,LicenseFee,CertificateNumber,LicenseStatus,LicenseeFirstName,...,CorpName,EstablishmentDesc,DayPerWeek,Description,AddrStreet,AddrZip,Location 1,2010 Census Wards Precincts,2010 Census Neighborhoods,Zip Codes
0,LBD7,BWL,333.0,11/25/2003,04/30/2004,2003,100.0,T070,Transferred,RONALD F.,...,"Z AND J, LLC",Tavern,7 days,"Beer, Wine, & Liquor",1633 CHARLES STREET SOUTH,21230,"1633 CHARLES STREET SOUTH\nBaltimore, MD 21230...",173.0,222.0,27953.0
1,LB,BWL,84.0,05/01/2003,04/30/2004,2003,1100.0,0584,Renewed,MICHAEL F,...,"DFVS ST.PAUL, LLC",Restaurant,7 days,"Beer, Wine, & Liquor",3103-05 ST. PAUL STREET,21218,"3103-05 ST. PAUL STREET\nBaltimore, MD 21218\n...",35.0,277.0,27307.0
2,LA,BWL,188.0,05/01/2003,04/30/2004,2003,715.0,0374,Renewed,JUNG J.,...,"DHK, INC.",Package goods only,6 days,"Beer, Wine, & Liquor",631 WASHINGTON BOULEVARD,21230,"631 WASHINGTON BOULEVARD\nBaltimore, MD 21230\...",1.0,194.0,27953.0
3,LC,BWL,31.0,05/01/2003,04/30/2004,2003,550.0,0831,Renewed,GENEVA,...,"EASTSIDE REINDEER ASSOCIATION, INC.",Non-Profits only,7 days,"Beer, Wine, & Liquor",1228 PRESTON STREET EAST,21202,"1228 PRESTON STREET EAST\nBaltimore, MD 21202\...",107.0,180.0,13645.0
4,LB,BWL,145.0,05/01/2003,04/30/2004,2003,1100.0,0649,Renewed,ANNE D.,...,"PATRICK'S OF PRATT STREET, INC.",Restaurant,7 days,"Beer, Wine, & Liquor",131 SCHROEDER STREET SOUTH,21223,"131 SCHROEDER STREET SOUTH\nBaltimore, MD 2122...",7.0,114.0,27632.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18161,LBD7,BWL,277.0,05/01/2017,04/30/2018,2017,1320.0,0834,Renewed,KURT X.,...,"KURT'S PLACE, LLC",Tavern,7 days,"Beer, Wine, & Liquor",601 CLINTON STREET SOUTH,21224,"601 CLINTON STREET SOUTH\nBaltimore, MD 21224\...",130.0,30.0,27935.0
18162,LA,BWL,215.0,05/01/2017,04/30/2018,2017,858.0,1204,Renewed,JANICE,...,"SUN BEVERAGE, INC.",Package goods only,6 days,"Beer, Wine, & Liquor",1532 HAVENWOOD ROAD,21218,"1532 HAVENWOOD ROAD\nBaltimore, MD 21218\n(39....",212.0,111.0,27307.0
18163,LB,BWL,246.0,05/01/2017,04/30/2018,2017,1520.0,0738,Renewed,ASMA K.,...,"ABM SYSTEMS, INC",Restaurant,7 days,"Beer, Wine, & Liquor",3105 ST. PAUL STREET,21218,"3105 ST. PAUL STREET\nBaltimore, MD 21218\n(39...",35.0,277.0,27307.0
18164,LBD7,BWL,401.0,05/01/2017,04/30/2018,2017,1820.0,1143,Renewed,ERIC D.,...,"KALI'S COURT, LLC",Tavern,7 days,"Beer, Wine, & Liquor",1602-08 THAMES STREET,21231,"1602-08 THAMES STREET\nBaltimore, MD 21231\n(3...",286.0,68.0,27954.0


In [25]:
# Listing the columns

df.columns

Index(['LicenseClass', 'SubClass', 'LicenseNumber', 'LicenseDate',
       'LicenseEndDate', 'LicenseYear', 'LicenseFee', 'CertificateNumber',
       'LicenseStatus', 'LicenseeFirstName', 'LicenseeLastName', 'TradeName',
       'CorpName', 'EstablishmentDesc', 'DayPerWeek', 'Description',
       'AddrStreet', 'AddrZip', 'Location 1', '2010 Census Wards Precincts',
       '2010 Census Neighborhoods', 'Zip Codes'],
      dtype='object')

In [26]:
# Creating a new df for dropped columns

df_dropped = df.drop(['LicenseClass', 'SubClass', 'LicenseDate', 'LicenseEndDate', 'CertificateNumber', 'LicenseStatus',
        'LicenseeFirstName', 'LicenseeLastName', 'TradeName', 'CorpName', 'EstablishmentDesc', 'DayPerWeek',
        'Description', 'AddrStreet', 'Location 1'], axis=1)

In [27]:
df_dropped

,LicenseNumber,LicenseYear,LicenseFee,AddrZip,2010 Census Wards Precincts,2010 Census Neighborhoods,Zip Codes
0,333.0,2003,100.0,21230,173.0,222.0,27953.0
1,84.0,2003,1100.0,21218,35.0,277.0,27307.0
2,188.0,2003,715.0,21230,1.0,194.0,27953.0
3,31.0,2003,550.0,21202,107.0,180.0,13645.0
4,145.0,2003,1100.0,21223,7.0,114.0,27632.0
...,...,...,...,...,...,...,...
18161,277.0,2017,1320.0,21224,130.0,30.0,27935.0
18162,215.0,2017,858.0,21218,212.0,111.0,27307.0
18163,246.0,2017,1520.0,21218,35.0,277.0,27307.0
18164,401.0,2017,1820.0,21231,286.0,68.0,27954.0


In [28]:
df_dropped.AddrZip.nunique()

26

In [29]:
df_dropped.LicenseYear.nunique()

15

In [11]:
#df_dropped = pd.get_dummies(df_dropped, columns = ['LicenseYear'])
#df_dropped

In [30]:
new_df = pd.DataFrame(data=np.c_[df_dropped.drop(['Zip Codes'], axis=1), df_dropped['Zip Codes']], columns= list(df_dropped.drop(['Zip Codes'], axis=1)) + ['Zip Codes'])

In [31]:
new_df

,LicenseNumber,LicenseYear,LicenseFee,AddrZip,2010 Census Wards Precincts,2010 Census Neighborhoods,Zip Codes
0,333.0,2003.0,100.0,21230.0,173.0,222.0,27953.0
1,84.0,2003.0,1100.0,21218.0,35.0,277.0,27307.0
2,188.0,2003.0,715.0,21230.0,1.0,194.0,27953.0
3,31.0,2003.0,550.0,21202.0,107.0,180.0,13645.0
4,145.0,2003.0,1100.0,21223.0,7.0,114.0,27632.0
...,...,...,...,...,...,...,...
18161,277.0,2017.0,1320.0,21224.0,130.0,30.0,27935.0
18162,215.0,2017.0,858.0,21218.0,212.0,111.0,27307.0
18163,246.0,2017.0,1520.0,21218.0,35.0,277.0,27307.0
18164,401.0,2017.0,1820.0,21231.0,286.0,68.0,27954.0


In [32]:
# Running the regression using sklearn with a test-train split

import math
train_df = new_df.iloc[0:math.ceil(0.8*new_df.shape[0])]
test_df = new_df.iloc[math.ceil(0.8*new_df.shape[0]):]

In [33]:
from sklearn import preprocessing
nm = preprocessing.Normalizer()
df_nm = nm.fit_transform(new_df)

In [34]:
df_nm

array([[9.47072309e-03, 5.69665416e-02, 2.84406099e-03, ...,
        4.92022551e-03, 6.31381539e-03, 7.95000368e-01],
       [2.42367696e-03, 5.77931543e-02, 3.17386269e-02, ...,
        1.00986540e-03, 7.99236332e-03, 7.87896986e-01],
       [5.34600425e-03, 5.69576942e-02, 2.03318778e-02, ...,
        2.84361928e-05, 5.51662140e-03, 7.94876897e-01],
       ...,
       [7.09433839e-03, 5.81678070e-02, 4.38349364e-02, ...,
        1.00935709e-03, 7.98834038e-03, 7.87500400e-01],
       [1.13885101e-02, 5.72833540e-02, 5.16884999e-02, ...,
        8.12247855e-03, 1.93121868e-03, 7.93901277e-01],
       [8.24625324e-03, 5.73541131e-02, 3.75346699e-02, ...,
        3.92407913e-03, 3.58285486e-03, 7.94341670e-01]])

In [35]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
X = train_df.drop(['Zip Codes'], axis=1)
y = train_df['Zip Codes']
print(len(y))

14533


In [36]:
lr.normalize= True

In [37]:
lr.fit(X,y)

LinearRegression(normalize=True)

In [38]:
test_df

,LicenseNumber,LicenseYear,LicenseFee,AddrZip,2010 Census Wards Precincts,2010 Census Neighborhoods,Zip Codes
14533,166.0,2016.0,1320.0,21215.0,63.0,140.0,27295.0
14534,29.0,2016.0,1520.0,21201.0,96.0,151.0,13642.0
14535,94.0,2016.0,200.0,21230.0,169.0,137.0,27953.0
14536,8.0,2016.0,165.0,21211.0,20.0,125.0,14006.0
14537,56.0,2016.0,858.0,21202.0,193.0,180.0,13645.0
...,...,...,...,...,...,...,...
18161,277.0,2017.0,1320.0,21224.0,130.0,30.0,27935.0
18162,215.0,2017.0,858.0,21218.0,212.0,111.0,27307.0
18163,246.0,2017.0,1520.0,21218.0,35.0,277.0,27307.0
18164,401.0,2017.0,1820.0,21231.0,286.0,68.0,27954.0


In [39]:
from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test = train_test_split(train_df.drop(['Zip Codes'], axis=1), train_df['Zip Codes'], test_size=0.3, random_state=100)
lr = LinearRegression()
reg = lr.fit(x_train,y_train)
reg.score(x_test, y_test)

0.7527441373618422